In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import control as ct
from pytz import timezone

import sys
sys.path.append('../../../')

from utils.plot import config_matplotlib, figsize, fig_save_and_show, plot_events_data
from utils.optimization import MyOptimizationProblem, convert_to_model_params, plot_optimization_params, plot_optimization_error, plot_compare, compute_metrics, estimate_polynomial_coefficient_bounds, save_model_params_to_json, load_model_params_from_json
from utils.data import get_events, get_sections, load_df, process_df, export_dataframe_to_latex
from utils.sensitivity_analysis import describe_param_with_uniform_distribution, describe_param_with_log_uniform_distribution, describe_param_with_triangular_distribution, describe_param_with_normal_distribution, describe_param_with_truncated_normal_distribution, describe_param_with_log_normal_distribution, wrapped_model_function, create_problem_spec_and_sample, plot_sampled_distribuitions, evaluate_problem, analyze_time_step, analyze_problem, sobol_sensitivity_analysis_from_model_params_to_outputs, plot_sensitivity_analysis, plot_sensitivity_analysis_heatmaps, get_region_mean, plot_sensitivity_analysis_bars

import warnings
warnings.filterwarnings("ignore")

config_matplotlib()
latex_img_path = '/home/joaoantoniocardoso/workspace_TCC/repo/thesis/assets/generated/'
latex_tex_path = '/home/joaoantoniocardoso/workspace_TCC/repo/thesis/tex/generated/'

# Sensitivity Analysis

In [ ]:
from model import Propulsion, get_hull_areas

Propulsion.build({})

In [6]:
motor_params_loaded = load_model_params_from_json('models/2020/motor/me0909_free_rotor_step_params.json')
esc_params_loaded = load_model_params_from_json('models/2020/esc/mam17_params.json')

hull_M = 293.7 # [kg]
hull_S_water, hull_S_air = get_hull_areas(hull_M, hull_cog_x=1.9760, hull_S_total=8.2379)

propulsion_params = motor_params_loaded | esc_params_loaded | {
    'rho_water': 1023,  # Estimated from public ocean dataset
    'rho_air': 1.1839,  # Estimated from public ocean dataset
    'esc_tau_fall': 0.5,  # Unknown
    'esc_tau_rise': 1.5,  # Unknown
    'trans_eta': 0.9,  # Arbitrary
    'trans_I_r_in': 0,  # Ignored
    'trans_I_r_out': 0,  # Ignored
    'trans_k': 14/22,  # Manually counted
    'prop_D': 9.0 * 0.0254,  # From manufacturer's specs
    'prop_k_T_coeffs': np.array([ 0.37208208, -0.10825004, -0.16735476,  0.02821403]),
    'prop_k_Q_coeffs': np.array([ 0.05470368, -0.01753569, -0.00050999, -0.01078306]),
    'prop_eta_R': 1.0,  # Unknown
    'prop_I_r': 5.54e-04,  # Unknown
    'hull_C_T': 0.01,  # Unknown
    'hull_M': hull_M,  # Measured (fully loaded, with pilot)
    'hull_M_a': 0,  # Unknown
    'hull_S_water': hull_S_water,  # Estimated from hydrostatics
    'hull_S_air': hull_S_air,  # Estimated from hydrostatics
    'hull_T_ded': 0.0855,  # Estimated from models/2020/propeller/propeller.ipynb
    'hull_W': 0.1425,  # Estimated from models/2020/propeller/propeller.ipynb
}
propulsion_params

{'motor_R_A': 0.019579524515654242,
 'motor_B': 0.0012793008044770459,
 'motor_K_Q': 0.10543193741528227,
 'motor_K_V': 0.10337093946599393,
 'motor_L_A': 6.336356266841513e-05,
 'motor_I_r': 0.011828773027862982,
 'rho_water': 1023,
 'rho_air': 1.1839,
 'esc_F_s': 12550,
 'esc_V_ds_ov': 2.0,
 'esc_R_ds_on': 0.008256754167097705,
 'esc_E_on': 7e-05,
 'esc_E_off': 0.00011,
 'esc_V_F': 3.338359854985,
 'esc_r_D': 0.004027165799996619,
 'esc_Q_rr': 1.9e-07,
 'esc_tau_fall': 0.5,
 'esc_tau_rise': 1.5,
 'trans_eta': 0.9,
 'trans_I_r_in': 0,
 'trans_I_r_out': 0,
 'trans_k': 0.6363636363636364,
 'prop_D': 0.2286,
 'prop_k_T_coeffs': array([ 0.37208208, -0.10825004, -0.16735476,  0.02821403]),
 'prop_k_Q_coeffs': array([ 0.05470368, -0.01753569, -0.00050999, -0.01078306]),
 'prop_eta_R': 1.0,
 'prop_I_r': 0.000554,
 'hull_C_T': 0.01,
 'hull_M': 293.7,
 'hull_M_a': 0,
 'hull_S_water': 4.142904990019334,
 'hull_S_air': 4.094995009980666,
 'hull_T_ded': 0.0855,
 'hull_W': 0.1425}

In [8]:
def model_function(T, U, X0, **params):
    return ct.input_output_response(
        Propulsion.build(params=params),
        T=T,
        U=U,
        X0=X0,
        solve_ivp_method='Radau',
        # solve_ivp_kwargs=dict(max_step=1),
    ).to_pandas()

def propulsion_sensitivity_analysis_step_response(
    params_description: dict,
    voltage_step: float,
    duty_cycle_step: float,
    samples: int,
    do_plot=True,
    calc_second_order=True,
):

    model_class = Propulsion
    model_params = propulsion_params
    model_tmp = model_class.build(model_params)

    # Time array
    T = np.linspace(0, 10, 11, endpoint=True)

    # Input data
    U = np.empty([model_tmp.ninputs, len(T)]) * np.nan # type: ignore ; obs: the "nan" here is to force us to define every input
    U[model_tmp.input_labels.index('batt_v')] = voltage_step
    U[model_tmp.input_labels.index('pilot_d')] = duty_cycle_step

    # Initial state
    X0 = model_class.initial_state(
        X0=dict(),
        U0=U[:,0],
        params=model_params,
    )

    df_sa, problem = sobol_sensitivity_analysis_from_model_params_to_outputs(
        params_description=params_description,
        T=T,
        U=U,
        X0=X0,
        model_function=model_function,
        model_class=model_class,
        model_params=model_params,
        samples=samples,
        calc_second_order=calc_second_order,
        seed=42,
        nprocs=32,
    )

    df_steady_state_mean = get_region_mean(df_sa, t_start=8, t_end=10)

    export_dataframe_to_latex(
        filename=f"{latex_tex_path}/propulsion_sensitivity_analysis.tex",
        label=f'table:propulsion_sensitivity_analysis',
        caption='Índices de Sobol em regime permanente',
        df_steady_state_mean=df_steady_state_mean,
    )

    if do_plot:
        display(df_steady_state_mean)

        # fig_save_and_show(
        #     None,
        #     'Distribuição dos valores para cada fator',
        #     'Distribution of the values for each factor',
        #     fig=plot_sampled_distribuitions(problem),
        #     ncols=3,
        # )

        # for output in df_sa['output'].unique():
        #     fig_save_and_show(
        #         None,
        #         f'Índices de Sobol para a saída {output} durante o tempo',
        #         f'Sobol Indices for the output {output} over time\nwith step of {duty_cycle_step*100} [\%] and {voltage_step} [V],\nwith {samples} samples',
        #         fig=plot_sensitivity_analysis(df_sa, output=output),
        #         ncols=3,
        #     )

        # for output in df_sa['output'].unique():
        #     fig_save_and_show(
        #         None,
        #         f'Índices de Sobol para a saída {output} durante o tempo',
        #         f'Sobol Indices for the output {output} over time\nwith step of {duty_cycle_step*100} [\%] and {voltage_step} [V],\nwith {samples} samples',
        #         fig=plot_sensitivity_analysis_heatmaps(df_sa, output=output),
        #         ncols=3,
        #     )

        for output in df_sa['output'].unique():
            fig_save_and_show(
                f'{latex_img_path}/propulsion_sensitivity_analysis_{output}.pdf',
                f'Índices de Sobol para a saída {output} em regime permanente',
                f'Sobol Indices for the output {output} at steady-state\nwith step of {duty_cycle_step*100} [%] and {voltage_step} [V],\nwith {samples} samples',
                fig=plot_sensitivity_analysis_bars(df_steady_state_mean, output=output),
                ncols=3,
            )

    return df_sa, problem


In [ ]:
# Setting all uncertainties to be x%
u_motor_R_A = 1/100
u_motor_L_A = 1/100
u_motor_B = 1/100
u_motor_I_r = 1/100
u_motor_K_Q = 1/100
u_prop_I_r = 1/100
u_prop_K_Q = 1/100
u_prop_D = 1/100
u_rho_water = 1/100
u_trans_eta = 1/100
u_esc_eta = 1/100
u_esc_tau_rise = 1/100
u_esc_tau_fall = 1/100
u_trans_K = 1/100

params_description = {
    'motor_R_A': describe_param_with_uniform_distribution(lower=propulsion_params['motor_R_A']*(1-u_motor_R_A), upper=propulsion_params['motor_R_A']*(1+u_motor_R_A)),
    'motor_L_A': describe_param_with_uniform_distribution(lower=propulsion_params['motor_L_A']*(1-u_motor_L_A), upper=propulsion_params['motor_L_A']*(1+u_motor_L_A)),
    'motor_B': describe_param_with_uniform_distribution(lower=propulsion_params['motor_B']*(1-u_motor_B), upper=propulsion_params['motor_B']*(1+u_motor_B)),
    'motor_I_r': describe_param_with_uniform_distribution(lower=propulsion_params['motor_I_r']*(1-u_motor_I_r), upper=propulsion_params['motor_I_r']*(1+u_motor_I_r)),
    'motor_K_Q': describe_param_with_uniform_distribution(lower=propulsion_params['motor_K_Q']*(1-u_motor_K_Q), upper=propulsion_params['motor_K_Q']*(1+u_motor_K_Q)),
    'prop_I_r': describe_param_with_uniform_distribution(lower=propulsion_params['prop_I_r']*(1-u_prop_I_r), upper=propulsion_params['prop_I_r']*(1+u_prop_I_r)),
    'prop_K_Q': describe_param_with_uniform_distribution(lower=propulsion_params['prop_K_Q']*(1-u_prop_K_Q), upper=propulsion_params['prop_K_Q']*(1+u_prop_K_Q)),
    'prop_D': describe_param_with_uniform_distribution(lower=propulsion_params['prop_D']*(1-u_prop_D), upper=propulsion_params['prop_D']*(1+u_prop_D)),
    'rho_water': describe_param_with_uniform_distribution(lower=propulsion_params['rho_water']*(1-u_rho_water), upper=propulsion_params['rho_water']*(1+u_rho_water)),
    'trans_eta': describe_param_with_uniform_distribution(lower=propulsion_params['trans_eta']*(1-u_trans_eta), upper=propulsion_params['trans_eta']*(1+u_trans_eta)),
    'esc_eta': describe_param_with_uniform_distribution(lower=propulsion_params['esc_eta']*(1-u_esc_eta), upper=propulsion_params['esc_eta']*(1+u_esc_eta)),
    'esc_tau_rise': describe_param_with_uniform_distribution(lower=propulsion_params['esc_tau_rise']*(1-u_esc_tau_rise), upper=propulsion_params['esc_tau_rise']*(1+u_esc_tau_rise)),
    'esc_tau_fall': describe_param_with_uniform_distribution(lower=propulsion_params['esc_tau_fall']*(1-u_esc_tau_fall), upper=propulsion_params['esc_tau_fall']*(1+u_esc_tau_fall)),
    'trans_k': describe_param_with_uniform_distribution(lower=propulsion_params['trans_k']*(1-u_trans_K), upper=propulsion_params['trans_k']*(1+u_trans_K)),
}

df_sa, problem = propulsion_sensitivity_analysis_step_response(
    params_description=params_description,
    voltage_step=36,
    duty_cycle_step=1,
    samples=2**14,
    do_plot=True,
    calc_second_order=False,
)